<a href="https://colab.research.google.com/github/mandemadhura/crewai_exercises/blob/main/event_planner_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determin

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [16]:
import os
#set openai API key
#set serper API key
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"

In [17]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

website_scrapper = ScrapeWebsiteTool()
serper_dev_tool = SerperDevTool()

In [28]:
venue_decider = Agent(
    role="Venue Coordintor",
    goal="Find and book an appropriate venue based on "
         "event requirements",
    backstory="You are a part of big event planner company. "
              "You have a good sense of space and event logistics. "
              "You are an expert at finding, negotiating and securing the "
              "space that would be best fit with the given budget, "
              "event size and the theme",
    verbose=True,
    tools=[website_scrapper, serper_dev_tool],
    allow_delegation=False
)

In [29]:
logistics_manager = Agent(
    role="Logistics Manager",
    goal="Handle all the logistics of the event "
         "based on event requirements.",
    backstory="You have a strong experties in handling "
              "event logistics such as catering, decoration "
              "and other aesthetics management flawlessly "
              "with the given event requirements and constraints.",
    tools=[serper_dev_tool, website_scrapper],
    verbose=True,
    allow_delegation=False
)

In [30]:
event_promoter = Agent(
    role="Event Promoter",
    goal="Effectively promote the event for a potantial future business growth",
    backstory="You work in a sales team of a big event planner firm. "
              "You are extraordinary at promoting any event with showcasing "
              "the qulity, scale and dedication of your team to make that event "
              "a big grand success for your customer. Your promotion shows the "
              "verstility of events that your company can handle.",
    verbose=True,
    tools=[serper_dev_tool, website_scrapper],
    allow_delegation=False
)

In [31]:
from pydantic import BaseModel

class VenueDetails(BaseModel):
  name: str
  address: str
  capacity: int
  booking_status: str

In [32]:
venue_task = Task(
    description="find a venue in the {city} for {expected_guests} people "
                   "that meets event criteria {event_topic}",
    expected_output="All the details of the venue chosen for that event",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
    agent=venue_decider
)

In [33]:
logistics_task = Task(
    description="Coordinate all the logistics, catering for {expected_guests} "
                "prticipants on {tentative_date}",
    expected_output="Confirmation of all the logistics arrangement.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [34]:
marketing_task = Task(
    description="Handle the responsibility to promote this event effectively",
    expected_output="Create a report that shows activities carried out "
                    "in the event and prticipants involvement in the "
                    "Markdown format",
    output_file="event_report.md",
    async_execution=True,
    agent=event_promoter
)

In [35]:
event_management_crew = Crew(
    agents=[venue_decider,
            logistics_manager,
            event_promoter],

    tasks=[venue_task,
           logistics_task,
           marketing_task],

    verbose=True
)

In [36]:
event_details = {
    'event_topic': "Wedding",
    'event_description': "A gathering of near and dear ones for a Wedding.",
    'city': "Sambhaji Nagar",
    'tentative_date': "2025-09-15",
    'expected_guests': 500,
    'budget': 200000,
    'venue_type': "Wedding Hall"
}

In [37]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordintor
 [INFO]: == Starting Task: find a venue in the Sambhaji Nagar for 500 people that meets event criteria Wedding


> Entering new CrewAgentExecutor chain...
I need to find a venue in Sambhaji Nagar for a wedding with a capacity of 500 people. I should start by researching potential venues in the area.

Action: Search the internet
Action Input: {"search_query": "wedding venues in Sambhaji Nagar for 500 people"} 


Search results: Title: AC Banquet Halls 401 To 500 Persons - Pune - Justdial
Link: https://www.justdial.com/Pune/AC-Banquet-Halls-401-To-500-Persons-in-Chhatrapati-Sambhaji-Nagar/nct-10892314
Snippet: Popular AC Banquet Halls 401 To 500 Persons in Chhatrapati Sambhaji Nagar, Pune · Sorina Hillside Resort · Shahi Banquets A.c. Hall · Dakkhan Banquet Hall · Dakkhan ...
---
Title: The 10 Best Wedding Venues in Aurangabad - Weddingwire.in
Link: https://www.weddingwire.in/wedding-venues/aurangabad
Snippet: The Meadows Resort. Anand Nagar, 